Эта тетрадка создаёт датасет из данных Пакендорф из wav файлов и таблицы spans.csv, в которой указаны границы фраз.

In [1]:
import os, re
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import warnings
from bs4 import XMLParsedAsHTMLWarning
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning, module='bs4')
from pydub import AudioSegment
import mutagen 
from mutagen.wave import WAVE 

In [64]:
spans = pd.read_csv('spans.csv')

In [65]:
spans['place'] = spans['filename'].apply(lambda x: x.split('__')[0])
spans['place'] = spans['place'].apply(lambda x: 'bystraja' if x.startswith('By') else 'topolinoe' if x.startswith('To') else 'sebian')

In [66]:
file_id = {}
for id, filename in enumerate(spans['filename'].unique()):
    file_id[filename] = str(id)
spans['file_id'] = spans['filename'].apply(lambda x: file_id[x])
spans.head()

,filename,phrase,checked,start,end,length,place,file_id
0,Bystraja District__Conversations__Adukanova_Ic...,Teːleŋni,тэлэӈни,18090,21025,2935,bystraja,0
1,Bystraja District__Conversations__Adukanova_Ic...,V armiju služili...,в армию служили,21025,24065,3040,bystraja,0
2,Bystraja District__Conversations__Adukanova_Ic...,Ọrọttịč teːleŋni,ороттыч тэлэӈни,34415,35575,1160,bystraja,0
3,Bystraja District__Conversations__Adukanova_Ic...,Tugenidu tụrkị...,тугэниду турки,108325,110685,2360,bystraja,0
4,Bystraja District__Conversations__Adukanova_Ic...,"Nu, a sobak obučat', kak vy eto delaete?",ну а собак обучать как вы это делаете,395194,403985,8791,bystraja,0


In [69]:
file_phrase = []
filenames = spans['filename'].unique()
for filename in tqdm(filenames):
    audio = AudioSegment.from_wav("../audio/"+filename)
    for ann_id, row in enumerate(spans[spans['filename'] == filename].iterrows()):
        row = row[1]
        file_num = '_000'[:-len(row['file_id'])]+row['file_id']
        ann_num = '_000'[:-len(str(ann_id+1))]+str(ann_id+1)
        newfilename = 'eve_bp_'+row['place']+file_num+ann_num+'.wav'
        
        newAudio = audio[row['start']:row['end']]
        newAudio.export('../dataset_pakendorf/'+newfilename, format="wav")
        
        file_phrase.append([filename, newfilename, row['checked']])

100%|███████████████████████████████████████████| 80/80 [01:48<00:00,  1.36s/it]


In [70]:
df = pd.DataFrame(file_phrase, columns = ['file_orig', 'file', 'text'])
df

,file_orig,file,text
0,Bystraja District__Conversations__Adukanova_Ic...,eve_bp_bystraja_000_001.wav,тэлэӈни
1,Bystraja District__Conversations__Adukanova_Ic...,eve_bp_bystraja_000_002.wav,в армию служили
2,Bystraja District__Conversations__Adukanova_Ic...,eve_bp_bystraja_000_003.wav,ороттыч тэлэӈни
3,Bystraja District__Conversations__Adukanova_Ic...,eve_bp_bystraja_000_004.wav,тугэниду турки
4,Bystraja District__Conversations__Adukanova_Ic...,eve_bp_bystraja_000_005.wav,ну а собак обучать как вы это делаете
...,...,...,...
8676,Topolinoe__Narratives__Personal Narratives__To...,eve_bp_topolinoe_079_060.wav,тачин упе укчэнэддын
8677,Topolinoe__Narratives__Personal Narratives__To...,eve_bp_topolinoe_079_061.wav,тара
8678,Topolinoe__Narratives__Personal Narratives__To...,eve_bp_topolinoe_079_062.wav,тар
8679,Topolinoe__Narratives__Personal Narratives__To...,eve_bp_topolinoe_079_063.wav,эвэдыч


In [71]:
df[['file', 'text']].to_csv('even_speech_pakendorf.csv', index=False)